In [1]:
import tensorflow as tf
import numpy as np
import random

In [2]:
EPOCHS = 100

#모델 정의

In [3]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel,self).__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(1024,activation='relu')
        self.dense2 = tf.keras.layers.Dense(1,activation = 'sigmoid')

    def __call__(self,x,training = False,mask = None):
        x = self.flatten(x)
        x = self.dense1(x)
        return self.dense2(x)

#데이터셋(불균형한 데이터셋으로 만듬)

In [4]:
cifar = tf.keras.datasets.cifar10

(x_train,y_train),(x_test,y_test) = cifar.load_data()
x_train, x_test = x_train/255.0,x_test/255.0

x_train_small = list()
y_train_small = list()
for x,y in zip(x_train,y_train):
    if (y == 0 and random.randint(0,100)<10) or y == 1:#0번 10퍼센트만 사용 1번 전부 나머진 ㄴㄴ
        x_train_small.append(x[:])#미리 flatten
        y_train_small.append(y)

x_test_small = list()
y_test_small = list()
for x,y in zip(x_test,y_test):
    if (y == 0 or y == 1):#0번 10퍼센트만 사용 1번 전부 나머진 ㄴㄴ
        x_test_small.append(x[:])#미리 flatten
        y_test_small.append(y)

x_train = np.stack(x_train_small, axis = 0)
y_train = np.stack(y_train_small, axis = 0)
 
x_test = np.stack(x_test_small, axis = 0)
y_test = np.stack(y_test_small, axis = 0)

train_ds = tf.data.Dataset.from_tensor_slices((x_train,y_train)).shuffle(10000).batch(32).prefetch(2048)
test_ds = tf.data.Dataset.from_tensor_slices((x_test,y_test)).batch(32).prefetch(2048)


170500096/170498071 [==============================] - 2s 0us/step


#Keras API 모델 학습(불균형한 데이터셋)

In [5]:
model = MyModel()
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy',
                       tf.keras.metrics.Precision(name='precision'),#shallow nn 이라 좀 보조해줄거
                       tf.keras.metrics.Recall(name='recall')])
model.fit(train_ds, validation_data = test_ds, epochs = EPOCHS)
#loss 존나 안낮아짐.

Epoch 1/100

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

172/172 [==============================] - 2s 11ms/step - loss: 0.4241 - accuracy: 0.9063 - precision: 0.9246 - recall: 0.9768 - val_loss: 1.0191 - val_accuracy: 0.5575 - val_precision: 0.5308 - val_recall: 0.9910
Epoch 2/100
172/172 [==============================] - 2s 9ms/step - loss: 0.2403 - accuracy: 0.9202 - precision: 0.9325 - recall: 0.9836 - val_loss: 1.2252 - val_accuracy: 0.5425 - val_precision: 0.5223 - val_recall: 0.9950
Epoch 3/100
172/172 [==============================] - 2s

#불균형한 데이터셋을 Borderline SMOTE사용하여 맞춰준다.

In [6]:
 pip install imblearn

In [7]:
from imblearn.over_sampling import BorderlineSMOTE

In [8]:
smote = BorderlineSMOTE()

x_train = x_train.reshape((x_train.shape[0], x_train.shape[1] * x_train.shape[2] * x_train.shape[3])).astype(np.float32)
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1] * x_test.shape[2] * x_test.shape[3])).astype(np.float32)

x_train, y_train = smote.fit_resample(x_train,y_train)


train_ds = tf.data.Dataset.from_tensor_slices((x_train,y_train)).shuffle(10000).batch(32).prefetch(2048)
test_ds = tf.data.Dataset.from_tensor_slices((x_test,y_test)).batch(32).prefetch(2048)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=F

In [9]:
model = MyModel()
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy',
                       tf.keras.metrics.Precision(name='precision'),#shallow nn 이라 좀 보조해줄거
                       tf.keras.metrics.Recall(name='recall')])
model.fit(train_ds, validation_data = test_ds, epochs = EPOCHS)
#loss가 존나 낮아지면서 학습이 잘 되는걸 확인할 수 있다.

Epoch 1/100
313/313 [==============================] - 3s 9ms/step - loss: 0.5733 - accuracy: 0.7518 - precision: 0.7711 - recall: 0.7162 - val_loss: 0.4514 - val_accuracy: 0.7795 - val_precision: 0.8542 - val_recall: 0.6740
Epoch 2/100
313/313 [==============================] - 2s 8ms/step - loss: 0.3964 - accuracy: 0.8249 - precision: 0.8520 - recall: 0.7864 - val_loss: 0.4384 - val_accuracy: 0.7920 - val_precision: 0.8724 - val_recall: 0.6840
Epoch 3/100
313/313 [==============================] - 2s 8ms/step - loss: 0.3138 - accuracy: 0.8707 - precision: 0.9053 - recall: 0.8280 - val_loss: 0.4636 - val_accuracy: 0.8125 - val_precision: 0.7692 - val_recall: 0.8930
Epoch 4/100
313/313 [==============================] - 2s 8ms/step - loss: 0.2618 - accuracy: 0.8992 - precision: 0.9320 - recall: 0.8612 - val_loss: 0.5038 - val_accuracy: 0.8125 - val_precision: 0.7585 - val_recall: 0.9170
Epoch 5/100
313/313 [==============================] - 2s 8ms/step - loss: 0.2335 - accuracy: 0.9112